# 学習、テスト

学習とテストを分ける。 8 : 2  
学習をcross validationし、ハイパーパラメーターを決める。  
そのハイパーパラメーターを使って、学習データすべてで学習する。
テストする。

# hold out

In [ ]:
# 1回分割
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(n_splits=1,      # 分割を1個生成
                  train_size=0.5,  # 学習は半分
                  test_size =0.5,  # テストも半分
                  random_state=0)  # 乱数種（再現用）
train_index, test_index = next(ss.split(X)) 
X_train, X_test = X[train_index], X[test_index] # 学習データ，テストデータ
y_train, y_test = y[train_index], y[test_index] # 学習データのラベル，テストデータのラベル

# 複数回分割
ss = ShuffleSplit(n_splits=10, 
                  train_size=0.5, 
                  test_size=0.5)
for train_index, test_index in ss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))

# 層の割合を維持して分割
from sklearn.model_selection import StratifiedShuffleSplit
ss = StratifiedShuffleSplit(n_splits=1, 
                            train_size=0.95, 
                            test_size=0.05, 
                            random_state=0)
train_index, test_index = next(ss.split(X, y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

## cross validation

In [ ]:
from sklearn.model_selection import KFold
ss = KFold(n_splits=10, shuffle=True)
for train_index, test_index in ss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

# 層の割合を維持して分割
from sklearn.model_selection import StratifiedKFold
ss = StratifiedKFold(n_splits=10, shuffle=True) # シャッフルしてから分割。
for train_index, test_index in ss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

# scoreまで一気に。
from sklearn.model_selection import cross_val_score
ave_score = cross_val_score(clf, X, y, cv=10) # StratifiedKFold
# ave_scoreは、10個のスコアリスト。
print("{0:4.2f} +/- {1:4.2f} %".format(ave_score.mean() * 100, ave_score.std() * 100))

# leave one out
from sklearn.model_selection import LeaveOneOut
loocv = LeaveOneOut()
train_index, test_index = next(loocv.split(X, y))
# scoreまで一気に。
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X, y,cv=loocv) # LeaveOneOut

# leave p out
from sklearn.model_selection import LeavePOut
lpocv = LeavePOut(2)
scores = cross_val_score(clf, X, y,cv=lpocv)

# leave one-group out
from sklearn.model_selection import LeaveOneGroupOut
logocv = LeaveOneGroupOut()
group = np.array(list(range(50))*12)
group = np.sort(group[:y.size])
for train_index, test_index in logocv.split(X, y, group):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf,X, y,groups=group,cv=logocv) # LeaveOneGroupOut

# ハイパーパラメータの調整